# Decorators

## ARK Meeting Tutorial

### March 19, 2019

#### Judit Ács

## Let's create a greeter function

- takes another function as a parameter
- greets the caller before calling the function

In [ ]:
def greeter(func):
    print("Hello")
    func()
    
def say_something():
    print("Let's learn some Python.")
    
greeter(say_something)
# greeter(12)

## Functions are first class objects

- they can be passed as arguments
- they can be returned from other functions (example later)

## Let's create a `count_predicate` function

- takes a iterable and a predicate (yes-no function)
- calls the predicate on each element
- counts how many times it returns True
- same as `std::count_if` in C++

In [ ]:
def count_predicate(predicate, iterable):
    true_count = 0
    for element in iterable:
        if predicate(element) is True:
            true_count += 1
    return true_count

## Q. Can you write this function in fewer lines?

### The predicate parameter

- it can be anything 'callable'

#### 1. function

In [ ]:
def is_even(number):
    return number % 2 == 0

numbers = [1, 3, 2, -5, 0, 0]

count_predicate(is_even, numbers)

#### 2. instance of a class that implements `__call__` (functor)

In [ ]:
class IsEven(object):
    def __call__(self, number):
        return number % 2 == 0
    
print(count_predicate(IsEven(), numbers))

IsEven()(123)
i = IsEven()
i(11)

#### 3. lambda expression

In [ ]:
count_predicate(lambda x: x % 2 == 0, numbers)

## Functions can be nested

In [ ]:
def parent():
    print("I'm the parent function")
    
    def child():
        print("I'm the child function")
        
parent()

the nested function is only accessible from the parent

In [ ]:
def parent():
    print("I'm the parent function")
    
    def child():
        print("I'm the child function")
    
    print("Calling the nested function")
    child()
        
parent()
# parent.child  # raises AttributeError

## Functions can be return values

In [ ]:
def parent():
    print("I'm the parent function")
    
    def child():
        print("I'm the child function")
        
    return child

child_func = parent()

print("Calling child")
child_func()


print("\nUsing parent's return value right away")
parent()()

## Closure: nested functions have access to the parent's scope

In [ ]:
def parent(value):
    
    def child():
        print("I'm the nested function. "
              "The parent's value is {}".format(value))
        
    return child
        
child_func = parent(42)

print("Calling child_func")
child_func()

In [ ]:
f1 = parent("abc")
f2 = parent(123)

f1()
f2()

f1 is f2

## Function factory

In [ ]:
def make_func(param):
    value = param
    
    def func():
        print("I'm the nested function. "
              "The parent's value is {}".format(value))
        
    return func

func_11 = make_func(11)
func_abc = make_func("abc")

func_11()
func_abc()

## Wrapper function factory

- let's create a function that takes a function return an almost identical function
- the returned function adds some logging

In [ ]:
def add_noise(func):
    
    def wrapped_with_noise():
        print("Calling function {}".format(func.__name__))
        func()
        print("{} finished.".format(func.__name__))
        
    return wrapped_with_noise

### Wrapping a function

The function we are going to wrap:

In [ ]:
def noiseless_function():
    print("This is not noise")
    
noiseless_function()

#### now add some noise

In [ ]:
noisy_function = add_noise(noiseless_function)

noisy_function()

#### Bound the original reference to the wrapped function

- i.e. `greeter` should refer to the wrapped function
- we don't need the original function

In [ ]:
def greeter():
    print("Hello")
    
print(id(greeter))
   
greeter = add_noise(greeter)
greeter()
print(id(greeter))

#### this turns out to be a frequent operation

In [ ]:
def friendly_greeter():
    print("Hello friend")
    
def rude_greeter():
    print("Hey you")
    
friendly_greeter = add_noise(friendly_greeter)
rude_greeter = add_noise(rude_greeter)
friendly_greeter()

rude_greeter()

## Decorator syntax

- a decorator is a function
  - that takes a function as an argument
  - returns a wrapped version of the function
- the decorator syntax is just syntactic sugar (shorthand) for:

```python
func = decorator(func)
```

In [ ]:
@add_noise
def informal_greeter():
    print("Yo")
    
# same as:
# informal_greeter = add_noise(informal_greeter)
    
informal_greeter()

### Pie syntax

- introduced in [PEP318](https://www.python.org/dev/peps/pep-0318/) in Python 2.4
- various syntax proposals were suggested, summarized [here](https://wiki.python.org/moin/PythonDecorators#A1._pie_decorator_syntax)

# Problem 1. Function metadata is lost

In [ ]:
informal_greeter.__name__

### Solution 1. Copy manually

In [ ]:
def add_noise(func):
    
    def wrapped_with_noise():
        print("Calling {}...".format(func.__name__))
        func()
        print("{} finished.".format(func.__name__))
        
    wrapped_with_noise.__name__ = func.__name__
    return wrapped_with_noise

@add_noise
def greeter():
    """meaningful documentation"""
    print("Hello")
    
print(greeter.__name__)

What about other metadata such as the docstring?

In [ ]:
print(greeter.__doc__)

### Solution 2. `functools.wraps`

In [ ]:
from functools import wraps

def add_noise(func):
    
    @wraps(func)
    def wrapped_with_noise():
        print("Calling {}...".format(func.__name__))
        func()
        print("{} finished.".format(func.__name__))
        
    return wrapped_with_noise

@add_noise
def greeter():
    """function that says hello"""
    print("Hello")
    
print(greeter.__name__)
print(greeter.__doc__)

## Problem 2. Function arguments

- so far we have only decorated functions without parameters
- to wrap arbitrary functions, we need to capture a variable number of arguments
- remember `args` and `kwargs`

In [ ]:
def function_with_variable_arguments(*args, **kwargs):
    print(args)
    print(kwargs)
    
function_with_variable_arguments(1, "apple", tree="peach", color="red")

#### the same mechanism can be used in decorators

In [ ]:
def add_noise(func):
    
    @wraps(func)
    def wrapped_with_noise(*args, **kwargs):
        print("Calling {}...".format(func.__name__))
        func(*args, **kwargs)
        print("{} finished.".format(func.__name__))
        
    return wrapped_with_noise

- the decorator has only one parameter: `func`, the function to wrap
- the returned function (`wrapped_with_noise`) takes arbitrary parameters: `args`, `kwargs`
- it calls `func`, the decorator's argument with arbitrary parameters

In [ ]:
@add_noise
def personal_greeter(name):
    print("Hello {}".format(name))
    
personal_greeter("John")
personal_greeter("oli")

# Decorators can take parameters too

- they have to return a decorator without parameters - decorator factory

In [ ]:
def decorator_with_param(param1, param2=None):
    
    print("Creating a new decorator: {0}, {1}".format(
        param1, param2))
    
    def actual_decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print("Wrapper function {}".format(
                func.__name__))
            print("Params: {0}, {1}".format(param1, param2))
            return func(*args, **kwargs)
        return wrapper
    
    return actual_decorator

In [ ]:
@decorator_with_param(42, "abc")
def personal_greeter(name):
    print("Hello {}".format(name))
    
@decorator_with_param(4)
def personal_greeter2(name):
    print("Hello {}".format(name))
    
print("\nCalling personal_greeter")
personal_greeter("Mary")

print("\nCalling personal_greeter2")
personal_greeter2("Harry")

In [ ]:
def hello(name):
    print("Hello {}".format(name))
    
hello = decorator_with_param(1, 2)(hello)
hello("john")

# Decorators can be implemented as classes

- `__call__` implements the wrapped function

In [ ]:
class MyDecorator(object):
    def __init__(self, func):
        self.func_to_wrap = func
        print("Function to wrap: {}".format(func.__name__))
        wraps(func)(self)
        
    def __call__(self, *args, **kwargs):
        print("before func {}".format(self.func_to_wrap.__name__))
        res = self.func_to_wrap(*args, **kwargs)
        print("after func {}".format(self.func_to_wrap.__name__))
        return res
    
@MyDecorator
def foo():
    print("bar")

print("\nCalling foo")
foo()

In [ ]:
@MyDecorator
def bar(name):
    print("Hi {}".format(name))
    
bar("John")

# See also

Decorator overview with some advanced techniques: https://www.youtube.com/watch?v=9oyr0mocZTg

A very deep dive into decorators: https://www.youtube.com/watch?v=7jGtDGxgwEY